In [ ]:
train_data_path = '/content/drive/MyDrive/reSkinCancerDetection/SkinCD_re/train'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet import ResNet50, preprocess_input
from keras.optimizers import gradient_descent_v2
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [ ]:
datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                             validation_split=0.2)

In [ ]:
train_datagen = datagen.flow_from_directory(train_data_path,
                                            class_mode='binary',
                                            target_size=(224, 224),
                                            batch_size=20,
                                            subset='training')

Found 5276 images belonging to 2 classes.


In [ ]:
validation_datagen = datagen.flow_from_directory(train_data_path,
                                                 class_mode='binary',
                                                 target_size=(224, 224),
                                                 batch_size=20,
                                                 subset='validation')

Found 1318 images belonging to 2 classes.


In [ ]:
base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

94781440/94765736 [==============================] - 1s 0us/step


In [ ]:
for layer in base_model.layers:
  layer.trainable = True

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Input, Conv2D, MaxPooling2D, Dropout

In [ ]:
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(256, 'relu'))
model.add(Dropout(0.4))
model.add(Dense(64, 'relu'))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dropout (Dropout)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 256)               25690368  
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                16448     
                                                                 
 dropout_2 (Dropout)         (None, 64)                0

In [ ]:
optimizer = gradient_descent_v2.SGD(lr=0.001, decay=1e-6, momentum=0.1, nesterov=True)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics='accuracy')

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [ ]:
early_stop = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=3, restore_best_weights=True)

#reducing learning rate on plateau
rlrop = ReduceLROnPlateau(monitor='val_accuracy', mode='max', patience= 3, factor= 0.5, min_lr= 1e-7, verbose=1)

# ModelCheck = ModelCheckpoint('/content/drive/MyDrive/reSkinCancerDetection/vgg19Models/model_today_aug-{epoch:02d}-{val_accuracy:.4f}.hdf5',
#                             monitor='val_loss',
#                             mode='min',
#                             save_best_only=True,
#                             verbose=1)
callbacks_list = [rlrop]

In [ ]:
batch_size = 20 
epochs = 50

history = model.fit(
    train_datagen,
    steps_per_epoch = train_datagen.samples // batch_size,
    validation_data = validation_datagen, 
    validation_steps = validation_datagen.samples // batch_size,
    epochs = epochs,
     callbacks=callbacks_list
    )

Epoch 1/50
263/263 [==============================] - 2555s 10s/step - loss: 0.5134 - accuracy: 0.7742 - val_loss: 0.3957 - val_accuracy: 0.8192 - lr: 0.0010
Epoch 2/50
263/263 [==============================] - 68s 257ms/step - loss: 0.3279 - accuracy: 0.8465 - val_loss: 0.3671 - val_accuracy: 0.8315 - lr: 0.0010
Epoch 3/50
263/263 [==============================] - 68s 258ms/step - loss: 0.2725 - accuracy: 0.8777 - val_loss: 0.3758 - val_accuracy: 0.8169 - lr: 0.0010
Epoch 4/50
263/263 [==============================] - 67s 256ms/step - loss: 0.2286 - accuracy: 0.9016 - val_loss: 0.3299 - val_accuracy: 0.8492 - lr: 0.0010
Epoch 5/50
263/263 [==============================] - 67s 256ms/step - loss: 0.1905 - accuracy: 0.9146 - val_loss: 0.3080 - val_accuracy: 0.8523 - lr: 0.0010
Epoch 6/50
263/263 [==============================] - 68s 259ms/step - loss: 0.1641 - accuracy: 0.9321 - val_loss: 0.3342 - val_accuracy: 0.8469 - lr: 0.0010
Epoch 7/50
263/263 [==============================] 

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
plt.plot(range(len(loss)), loss, 'y', label='Training loss')
plt.plot(range(len(val_loss)), val_loss, 'r', label='Validation loss')
plt.title('Training and validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
plt.plot(range(len(acc)), acc, 'y', label='Training acc')
plt.plot(range(len(val_acc)), val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [ ]:
test_generator = test_datagen.flow_from_directory(test_data_path,
                                                  target_size=(224, 224),
                                                  shuffle=False,
                                                  class_mode='binary',
                                                  batch_size=1)

filenames = test_generator.filenames
nb_samples = len(filenames)
y_true = test_generator.classes

In [ ]:
y_pred = model.predict(test_generator,steps = nb_samples)

In [ ]:
y_pred = y_pred > 0.5

In [ ]:
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_true, y_pred)
mat

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_true, y_pred))